In [1]:
'''
Steps:
Pixelate the city of LA
Place the current chargers and set them with a radius
Figure out set of potential locations
Single family, street corners (write some code to find these points)
Characterize the pixels with the most important features
Set weights for each pixel in the LA city grid 
Run CELF 
Have the algorithm account for the different pixel weights in calculating cover
'''

'\nSteps:\nPixelate the city of LA\nPlace the current chargers and set them with a radius\nFigure out set of potential locations\nSingle family, street corners (write some code to find these points)\nCharacterize the pixels with the most important features\nSet weights for each pixel in the LA city grid \nRun CELF \nHave the algorithm account for the different pixel weights in calculating cover\n'

# Place Current Chargers

In [2]:
from functools import partial

!pip install pyproj
import pyproj as pyproj
from shapely import geometry
from shapely.geometry import Point
from shapely.ops import transform

     |████████████████████████████████| 6.5MB 7.4MB/s 


In [3]:
def convert(coord, radius=400):  
  lat = coord[0]
  lon = coord[1]
  # radius = coord[2] if len(coord) == 3 else radius
  local_azimuthal_projection = "+proj=aeqd +R=6371000 +units=m +lat_0={} +lon_0={}".format(
      lat, lon
  )
  wgs84_to_aeqd = partial(
      pyproj.transform,
      pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
      pyproj.Proj(local_azimuthal_projection),
  )
  aeqd_to_wgs84 = partial(
      pyproj.transform,
      pyproj.Proj(local_azimuthal_projection),
      pyproj.Proj("+proj=longlat +datum=WGS84 +no_defs"),
  )

  center = Point(float(lon), float(lat))
  point_transformed = transform(wgs84_to_aeqd, center)
  buffer = point_transformed.buffer(radius)
  # Get the polygon with lat lon coordinates
  return transform(aeqd_to_wgs84, buffer)

In [4]:
# initial chargepoint dataset
import pandas as pd
url = 'https://raw.githubusercontent.com/medhivya/ev-charger/main/chargepoint_coords_w_radius.csv'
cp_data = pd.read_csv(url)

In [5]:
# convert already placed chargers into Polygon objects
circles = cp_data.apply(convert, axis=1).reset_index().drop(['index'], axis=1)
circles = circles[0]

In [6]:
# # find total covered area
small_cover = circles[0]
for i in range(len(circles)):
  small_cover = small_cover.union(circles[i])

In [7]:
small_cover.area

0.0011047638644819426

In [8]:
# additional NREL set
# CHANGE CODE HERE TO SWITCH OUT NEW SET
url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.csv?fuel_type=ELEC&state=CA&status=E&access=public&api_key=JDCZwYUVGgKuaIHzmjwrmYsCaKbXotIHg5dyr3bv'
nrel_chargers_data_raw = pd.read_csv(url, usecols=['City', 'Latitude', 'Longitude'])
nrel_chargers_data_la = nrel_chargers_data_raw[nrel_chargers_data_raw.City == 'Los Angeles']

# drops an errorneous charger point
nrel_chargers_data_la = nrel_chargers_data_la.sort_values(by=['Longitude'])
nrel_chargers_data_la.drop(nrel_chargers_data_la.tail(1).index,inplace=True)
nrel_chargers_data_la = nrel_chargers_data_la.drop(['City'], axis = 1).reset_index().drop(['index'], axis=1)

nrel_chargers_data_la

# this new set can be changed, needs to be a lat, long with proper index
new_point_list = nrel_chargers_data_la

In [9]:
# process the new set 
new_circles_raw = new_point_list.apply(convert, axis=1).reset_index().drop(['index'], axis=1)

In [10]:
new_circles = new_circles_raw[0]

In [11]:
# calculate total cover including NREL data
covered = small_cover
for i in range(len(new_circles)):
  covered = covered.union(new_circles[i])

In [12]:
covered.area

0.02046695506728996

# Find Valid Possible Points

In [13]:
import pandas as pd
import urllib.request
import json
from shapely.geometry import Point, Polygon

In [14]:
# Intersections in the city of LA
url = 'https://raw.githubusercontent.com/medhivya/ev-charger/main/Intersections.csv'
intersections = pd.read_csv(url, usecols=['LAT', 'LON'])
possible_points = [Point(float(row[1][0]), float(row[1][1])) for row in intersections.iterrows()]

In [15]:
# Multiple Family and Commercial Zones
zoning_url = 'https://services5.arcgis.com/7nsPwEMP38bSkCjy/arcgis/rest/services/GPLU/FeatureServer/0/query?where=CATEGORY%20%3D%20%27COMMERCIAL%27%20OR%20CATEGORY%20%3D%20%27MULTIPLE%20FAMILY%27&outFields=GPDESC,CATEGORY,Shape__Area,Shape__Length&outSR=4326&f=json'
with urllib.request.urlopen(zoning_url) as url:
    family_zones = json.loads(url.read().decode())   

Note: family_zones variable naming is an artifact of when this analysis was run for single family zone usage, all further references are to the zones that we are actually examining, Multi Family and Commercial 

In [16]:
# make Polygons out of the zones
family_zone_shapes = []
for shape in (family_zones['features']):
  bounds = shape['geometry']['rings'][0]
  coords = [(i[1], i[0]) for i in bounds]
  family_zone_shapes.append(Polygon(coords))

In [17]:
# because the world is not flat
import pyproj
from shapely.ops import transform
from functools import partial

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj('+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +no_defs'))

# family_zone_shapes_transform = [transform(project, poly) for poly in family_zone_shapes]
# possible_points_transform = [transform(project, pt) for pt in possible_points]

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [19]:
from tqdm.notebook import tqdm

In [20]:
# filter intersections by the correct zones
family_zone_pts = []
for poly in tqdm(family_zone_shapes):
  for pt in possible_points:
    if pt.within(poly):
      family_zone_pts.append(pt)
      possible_points.remove(pt)

In [21]:
# read in council districts
url = 'https://raw.githubusercontent.com/medhivya/ev-charger/main/Council%20Districts%20(1).geojson'
with urllib.request.urlopen(url) as url:
    council_districts = json.loads(url.read().decode())

In [22]:
# translate council districts to Polygons
council_district_shapes = []
for shape in (council_districts['features']):
  bounds = shape['geometry']['coordinates'][0][0]
  coords = [(i[1], i[0]) for i in bounds]
  # print(coords)
  council_district_shapes.append(Polygon(coords))

In [23]:
len(council_district_shapes)

15

In [24]:
# organize points by council district
council_district_organized = []
for idx, shape in tqdm(enumerate(council_district_shapes), total=len(council_district_shapes)):
    council_district_organized.append([pt for pt in family_zone_pts if pt.within(shape)])


In [25]:
# convert points in each council district to circles
council_district_organized_circles = []
council_district_mapping_pts = []
for district in tqdm(council_district_organized):
  if len(district) > 0: 
    converted = [(pt.x, pt.y) for pt in district]
    council_district_mapping_pts.append(converted)
    council_district_organized_circles.append(pd.DataFrame(converted).apply(convert, axis=1).reset_index().drop(['index'], axis=1)[0])
  else:
    council_district_organized_circles.append([None for _ in range(10)])
    council_district_mapping_pts.append(converted)

# Run CELF

In [26]:
# CELF function
import heapq

def F(ind, new_circles, covered):
  return covered.union(new_circles[ind])

def celf(new_circles, k=10, covered = covered):
  heap = []
  for ind in range(len(new_circles)):
      heapq.heappush(heap, (F(ind, new_circles, covered).area * -1, ind))
  chosen_circles = []
  while len(chosen_circles) < k:
    top = heapq.heappop(heap)
    cvg = F(top[1], new_circles, covered)
    if len(heap) == 0:
          chosen_circles.append(top[1])
          break
    while ((cvg.area - covered.area) * -1) > heap[0][0]:
      heapq.heappush(heap, ((cvg.area - covered.area) * -1, top[1]))
      top = heapq.heappop(heap)
      cvg = F(top[1], new_circles, covered)
    chosen_circles.append(top[1])
    covered = cvg
  return chosen_circles

In [55]:
og_covered = covered # just in case?
results = []
for idx, council in tqdm(enumerate(council_district_organized_circles), total=len(council_district_organized_circles)):
  if None not in council: 
    res = celf(council, covered=og_covered)
    results.append([idx, res])
    print(idx, ':', res)


0 : [4, 6, 3, 5, 0, 1, 2]
1 : [3, 2, 0, 1]
2 : [44, 39, 11, 41, 1, 43, 40, 6, 0, 10]
3 : [1, 0]
5 : [3, 12, 15, 7, 1, 11, 2, 4, 13, 8]
6 : [0, 38, 19, 30, 3, 34, 26, 32, 23, 13]
7 : [0]
8 : [0]
11 : [50, 11, 42, 23, 78, 67, 65, 4, 64, 60]
13 : [4, 0, 5, 1, 3, 2]
14 : [4, 26, 19, 0, 9, 1, 25, 18, 20, 7]



In [48]:
# map res back to objects
res_circles = []
for row in enumerate(results):
  res_circles.append([council_district_mapping_pts[row[1][0]][i] for i in row[1][1]])

# Visualize Results

In [49]:
import folium
import json
from folium import plugins

In [50]:
import urllib.request

url = 'https://raw.githubusercontent.com/ritvikmath/StarbucksStoreScraping/master'
la_map_url = f'{url}/laMap.geojson'
with urllib.request.urlopen(la_map_url) as url:
    la_area = json.loads(url.read().decode())

In [51]:
la_map_rad = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=9)
folium.GeoJson(la_area).add_to(la_map_rad)
for i,row in cp_data.iterrows():
    folium.Circle((row.Latitude,row.Longitude), radius=1000, weight=2, color='red', fill_color='red', fill_opacity=.05).add_to(la_map_rad)


In [52]:
for i,row in new_point_list.iterrows():
  folium.Circle((row.Latitude,row.Longitude), radius=1000, weight=2, color='red', fill_color='red', fill_opacity=.05).add_to(la_map_rad)

In [53]:
for idx, district in enumerate(res_circles):
  for circle in district:
      folium.Circle((circle[0],circle[1]), radius=1000, weight=2, color='blue', fill_color='blue', fill_opacity=.05).add_to(la_map_rad)

In [54]:
la_map_rad